In [46]:
import findspark
findspark.init('/home/yura0217/spark-2.3.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
#Start a simple Spark Session
spark = SparkSession.builder.appName('WarmUp').getOrCreate()

In [3]:
#Load the Walmart Stock CSV File, have Spark infer the data types
df = spark.read.csv('walmart_stock.csv',header=True,inferSchema = True)

In [4]:
#What are the column names?
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [5]:
#What does the Schema look like?
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [6]:
#Print out the first 5 columns.
for i in range(0,5):
    print()
    print(df.head(5)[i])


Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)

Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)

Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)

Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)


In [7]:
#Use describe() to learn about the DataFrame
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [11]:
#printSchema
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)
from pyspark.sql.types import StructType
data_schema = [StructField('summary', StringType(), True),
               StructField('Open', StringType(), True),
               StructField('High', StringType(), True),
               StructField('Low', StringType(), True),
               StructField('Close', StringType(), True),
               StructField('Volume', StringType(), True),
               StructField('Adj Close', StringType(), True),]
final_struc = StructType(fields = data_schema)
df1 = spark.read.csv('walmart_stock.csv',schema = final_struc, header=True,inferSchema = True)
df1.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [12]:
#New dataframe HV Ratio
df.withColumn('HVRatio', df['High']/df['Volume']).select('HVRatio').show()

+--------------------+
|             HVRatio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [45]:
#What day had the Peak High in Price?
df.orderBy(df['High'].desc()).head()['Date']

datetime.datetime(2015, 1, 13, 0, 0)

In [13]:
#What is the mean of the Close column? 
df.agg({'Close':'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [14]:
#What is the max and min of the Volume column? 
from pyspark.sql.functions import *
df.groupBy().agg(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [15]:
#How many days was the Close lower than 60 dollars?
df.filter('Close < 60').count()

81